# 신경망 모델 구성하기

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cpu device


## 클래스 정의하기

In [3]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2])


## 모델 계층(Layer)

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2904,  0.1444,  0.3027, -0.2686,  0.3834,  0.1680, -0.2062,  0.2734,
         -0.1779, -0.0782, -0.1169, -0.2219, -0.4155, -0.0555,  0.3161, -0.2506,
          0.4737,  0.2737,  0.3572, -0.0279],
        [-0.5793, -0.1304,  0.0323, -0.1194,  0.4753,  0.0071, -0.3905,  0.0266,
         -0.1806, -0.1005, -0.0191, -0.8706, -0.2888, -0.2283,  0.6967, -0.2332,
          0.1599,  0.0579, -0.1103,  0.0149],
        [-0.2855, -0.0601, -0.0113, -0.4544,  0.1370, -0.1836, -0.4265, -0.0511,
         -0.2101,  0.2749,  0.3557, -0.6446, -0.2922, -0.4830,  0.5506, -0.3255,
          0.4888,  0.0028,  0.3591, -0.0499]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1444, 0.3027, 0.0000, 0.3834, 0.1680, 0.0000, 0.2734, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3161, 0.0000, 0.4737, 0.2737,
         0.3572, 0.0000],
        [0.0000, 0.0000, 0.0323, 0.0000, 0.4753, 0.0071, 0.0000, 0.0266, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.69

### nn.Sequential

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 모델 매개변수

In [20]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[: 2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0172,  0.0249, -0.0328,  ...,  0.0301,  0.0252,  0.0108],
        [-0.0228, -0.0343, -0.0336,  ...,  0.0037, -0.0307,  0.0332]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0347, -0.0231], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0107,  0.0317, -0.0439,  ...,  0.0325, -0.0227, -0.0180],
        [-0.0394, -0.0293, -0.0299,  ...,  0.0012,  0.0011, -0.0076]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Siz